In [103]:
import re
import os

# === Step: Parse common_columns.md to get defined doc() keys ===
# This pulls out all doc block identifiers like {% docs my_column %} from the Markdown file
def parse_common_columns(md_path):
    doc_keys = set()
    doc_pattern = re.compile(r"\{\%\s*docs\s+([^\s\%]+)\s*\%\}")
    with open(md_path, encoding="utf-8") as f:
        for line in f:
            match = doc_pattern.search(line)
            if match:
                doc_keys.add(match.group(1))  # Save the column name
    return doc_keys

# Set your actual file path here
COMMON_COLUMNS_MD = r"C:\dbt-opt-automation-dbx\src\models\docs\common_columns.md"
common_cols = parse_common_columns(COMMON_COLUMNS_MD)
print(f"✅ Parsed {len(common_cols)} doc() keys from common_columns.md")


✅ Parsed 159 doc() keys from common_columns.md


In [104]:
import yaml
from collections import defaultdict

def generate_exec_doc_package(table_name, columns, common_cols, column_sources, table_descriptions,
    existing_exec_columns):

    """
    Build schema.yml-compatible column blocks for a table and track common doc usage.

    Args:
    - table_name: name of the dbt model
    - columns: list of column names for this table
    - common_cols: set of column names already defined in common_columns.md
    - column_sources: dict mapping col -> [(model_name, doc_path)]

    Returns:
    - schema_block: list of dicts to be included in schema.yml
    - new_common_entries: list of (col_name, definition) for potential new common defs
    - conflicts: list of columns with conflicting definitions to review manually
    - doc_block_placeholders: Jinja doc block templates for unresolved conflicts
    """
    table_description = table_descriptions.get(table_name, "")
    schema_block = []
    new_common_entries = []
    conflicts = []

    for col in columns:
        # Reference existing common documentation
        if col in common_cols:
            schema_block.append({
                "name": col,
                "description": f"{{{{ doc('{col}') }}}}"
            })
            continue

        # If column has been defined elsewhere, try to centralize or flag conflicts
        if col in column_sources:
            def_map = defaultdict(set)  # maps definition -> (model, filepath)

            # Parse each source file where this column appears
            for model_name, filepath in column_sources[col]:
                try:
                    with open(filepath, encoding="utf-8") as f:
                        doc_data = yaml.safe_load(f)

                    for m in doc_data.get("models", []):
                        for c in m.get("columns", []):
                            if c.get("name") == col:
                                desc = c.get("description", "")
                                def_map[desc.strip()].add((m.get("name"), filepath))
                                break
                except Exception:
                    continue

            # Filter out blank or already-variable definitions
            non_blank_defs = [
                desc for desc in def_map
                if desc and not desc.strip().startswith("{{ doc(")
            ]

            # If there's exactly one valid definition, treat it as reusable
            if len(set(non_blank_defs)) == 1:
                clean_def = non_blank_defs[0]
                schema_block.append({
                    "name": col,
                    "description": f"{{{{ doc('{col}') }}}}"
                })
                new_common_entries.append((col, clean_def))

            else:
                # Still reference as doc() in schema, but surface for manual review
                schema_block.append({
                    "name": col,
                    "description": f"{{{{ doc('{col}') }}}}"
                })

                conflict_items = []
                for desc, instances in def_map.items():
                    if not desc or desc.strip().startswith("{{ doc("):
                        continue
                    for model_name, filepath in instances:
                        conflict_items.append({
                            "column": col,
                            "model": model_name,
                            "file": filepath,
                            "definition": desc
                        })

                if conflict_items:
                    conflicts.append({
                        "column": col,
                        "instances": conflict_items
                    })


        else:
            # Fall back to any existing definition in executive YAML (only if non-blank)
            fallback_desc = existing_exec_columns.get(col, "").strip()
            schema_block.append({
                "name": col,
                "description": fallback_desc if fallback_desc else f"{{{{ doc('{col}') }}}}"
            })

    # Build placeholder {% docs %} blocks for conflicts
    doc_block_placeholders = []
    for conflict in conflicts:
        col = conflict["column"]
        definitions = sorted(set(i["definition"] for i in conflict["instances"]))
        models = sorted(set(i["model"] for i in conflict["instances"]))

        doc_lines = [
            f"# TODO: `{col}` has conflicting definitions across {len(models)} model(s).",
            f"# - Models involved: {', '.join(models)}",
            "# - Definitions include:"
        ]
        for d in definitions:
            doc_lines.append(f"#   - \"{d}\"")

        placeholder = "\n".join(doc_lines) + f"\n\n{{% docs {col} %}}\n{{% enddocs %}}"
        doc_block_placeholders.append(placeholder)

    return schema_block, new_common_entries, conflicts, doc_block_placeholders, table_description




In [105]:
def parse_existing_exec_columns(exec_doc_path):
    """
    Returns:
      {
        table_name: {
          column_name: description
        }
      }
    """
    with open(exec_doc_path, encoding="utf-8") as f:
        data = yaml.safe_load(f)

    table_map = {}

    for model in data.get("models", []):
        table = model.get("name")
        col_map = {}
        for col in model.get("columns", []):
            desc = col.get("description", "")
            if desc and desc.strip():
                col_map[col["name"]] = desc.strip()
        table_map[table] = col_map

    return table_map


In [107]:
import os

# === Step 1: Identify all SQL models in the Executive folder ===
EXECUTIVE_MODELS_PATH = r"C:\dbt-opt-automation-dbx\src\models\marts\executive"

def get_executive_table_names(path):
    """
    Recursively scans a dbt models folder to collect table names
    based on .sql filenames (lowercased, sans extension).
    """
    table_names = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith(".sql"):
                table_name = os.path.splitext(file)[0]
                table_names.append(table_name.lower())
    return sorted(set(table_names))

executive_tables = get_executive_table_names(EXECUTIVE_MODELS_PATH)

# === Step 2: Print WHERE clause for metadata query ===
print("=== 📄 Paste this SQL WHERE clause into your warehouse ===\n")
in_clause = "', '".join(executive_tables)
print(f"""
SELECT table_schema, table_name, column_name, ordinal_position
FROM information_schema.columns
WHERE table_name IN ('{in_clause}')
""")

# === Define CSV path for expected metadata export ===
CSV_PATH = r"C:\dbt-opt-automation-dbx\column_metadata_export.csv"

# === Instructions for manual metadata export ===
print("=== 📝 Instructions ===")
print("1. Paste and run the SQL query above in your data warehouse (e.g., Databricks).")
print("2. Export the results as a CSV file named `column_metadata_export.csv`.")
print("   This file is temporary and can be saved directly under `C:\\dbt-opt-automation-dbx`.")
print("   It should be deleted or excluded from any pushes to a shared repo (e.g., via `.gitignore`).")
print("3. The notebook will use this CSV to drive documentation generation and audit logic.\n")
print("📁 Expected file path:", CSV_PATH)

=== 📄 Paste this SQL WHERE clause into your warehouse ===


SELECT table_schema, table_name, column_name, ordinal_position
FROM information_schema.columns
WHERE table_name IN ('agg_cpsc_aqa', 'agg_cpsc_medlag', 'agg_cpsc_medlag_trend', 'agg_cpsc_npv', 'agg_cpsc_npv_trend', 'agg_cpsc_telehealth', 'agg_purchased_services', 'agg_qa_calc', 'agg_qa_domain_trend', 'agg_qa_trend', 'dim_cpsc_aqa_star', 'dim_cpsc_mbr', 'dim_cpsc_region', 'dim_pa_provider', 'dim_pa_system', 'dim_ps_categories_flat', 'dim_ps_category_savings', 'dim_ps_date_rank', 'dim_ps_level_one_cat', 'dim_ps_level_three_cat', 'dim_ps_level_two_cat', 'dim_ps_member', 'dim_ps_member_sys', 'dim_ps_savings_rates', 'dim_qa_domain', 'dim_qa_period', 'executive_report_nso', 'load_ps_categories', 'load_ps_member', 'load_purchased_services', 'upwr_rpt_er_provider', 'upwr_rpt_er_system', 'upwr_rpt_pa_sl', 'upwr_rpt_pa_sl_trend', 'upwr_rpt_pa_subsl', 'upwr_rpt_pa_subsl_trend', 'upwr_rpt_sg2_capacity', 'upwr_rpt_sg2_cmi', 'upwr_rpt_sg2_in

In [ ]:
import csv
from collections import defaultdict

# === Step 3: Load exported column metadata from warehouse ===

# Initialize storage for column lists per table
exec_cols = defaultdict(list)

# Read CSV and build dictionary of columns per table
with open(CSV_PATH, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        schema = row.get("table_schema")
        table = row.get("table_name")
        col = row.get("column_name")

        # Filter by expected schema; adjust if needed for different environments
        if schema == "optauto_assets" and table and col:
            exec_cols[table].append(col)

# Deduplicate and sort columns for consistency
exec_cols = {k: sorted(set(v)) for k, v in exec_cols.items()}


In [93]:
import yaml

def load_table_descriptions_from_yml(path):
    with open(path, encoding="utf-8") as f:
        data = yaml.safe_load(f)
    return {
        model["name"]: model.get("description", "")
        for model in data.get("models", [])
        if isinstance(model, dict) and "name" in model
    }


In [94]:
EXISTING_YML_PATH = r"C:\dbt-opt-automation-dbx\src\models\marts\executive\_executive_documentation.yml"
table_descriptions = load_table_descriptions_from_yml(EXISTING_YML_PATH)


In [95]:
import glob
import yaml
import os

# === Step 4: Scan documentation YAMLs to map columns to their definitions ===

def scan_column_definitions_from_docs(filepaths):
    col_to_sources = {}
    errors = []

    for filepath in filepaths:
        try:
            with open(filepath, encoding="utf-8") as f:
                data = yaml.safe_load(f)

            if not isinstance(data, dict) or "models" not in data:
                errors.append((filepath, "Missing 'models' block"))
                continue

            for model in data["models"]:
                model_name = model.get("name")
                for col in model.get("columns", []):
                    col_name = col.get("name")
                    if col_name:
                        col_to_sources.setdefault(col_name, []).append(
                            (model_name, filepath)
                        )

        except Exception as e:
            errors.append((filepath, str(e)))

    return col_to_sources, errors


# === Build documentation file list ===
ALL_DOCS = glob.glob(
    r"C:\dbt-opt-automation-dbx\src\models\marts\**\*_documentation.yml",
    recursive=True
)

EXEC_DOC_PATH = os.path.normpath(
    r"C:\dbt-opt-automation-dbx\src\models\marts\executive\_executive_documentation.yml"
)

ALL_DOCS = [os.path.normpath(p) for p in ALL_DOCS]

MARTS_DOCS_EXCLUDING_EXEC = [
    p for p in ALL_DOCS if p != EXEC_DOC_PATH
]

# === Execute scan (THIS is the only call) ===
column_sources, doc_errors = scan_column_definitions_from_docs(
    MARTS_DOCS_EXCLUDING_EXEC
)

# === Optional diagnostics ===
if doc_errors:
    print("⚠️ Errors in the following documentation files:")
    for path, msg in doc_errors:
        print(f"  - {path}: {msg}")
else:
    print("✅ All documentation files parsed successfully.")

existing_exec_columns = parse_existing_exec_columns(EXEC_DOC_PATH)

# Flatten and clean fallback column descriptions
existing_exec_columns = {
    col: desc.strip()
    for table_dict in existing_exec_columns.values()
    for col, desc in table_dict.items()
    if desc and desc.strip()
}


✅ All documentation files parsed successfully.


In [96]:
from collections import defaultdict

# === Step 5: Execute doc generation across all executive tables ===

# Initialize accumulators for output artifacts
full_schema_models = []               # For the final schema.yml block
all_new_common_docs = defaultdict(set)  # For new common column doc entries
all_conflicts = []                    # For review due to definition conflicts
all_placeholders = []                # For unresolved doc blocks needing manual input

# Iterate through each executive model table
for table in executive_tables:
    cols = exec_cols.get(table)
    if not cols:
        continue  # Skip tables missing column metadata

    schema, new_docs, review, placeholders, description = generate_exec_doc_package(
        table_name=table,
        columns=cols,
        common_cols=common_cols,
        column_sources=column_sources,
        table_descriptions=table_descriptions,
        existing_exec_columns=existing_exec_columns
    )


    # Append schema block for current table
    full_schema_models.append({
        "name": table,
        "description": description,
        "columns": schema
    })

    # Collect new common column entries (centralizable definitions)
    for col, desc in new_docs:
        all_new_common_docs[col].add(desc)

    # Collect conflicts and placeholders for manual follow-up
    all_conflicts.extend(review)
    all_placeholders.extend(placeholders)

# === Output: Full schema.yml documentation block ===
print("\n\n=== 📄 schema.yml block (models list) ===\n")
final_schema = {"models": full_schema_models}
print(yaml.dump(final_schema, sort_keys=False, default_flow_style=False))




=== 📄 schema.yml block (models list) ===

models:
- name: agg_cpsc_aqa
  description: Aggregate quality assessment scores for CPSC members, capturing rankings
    across access, cost, efficiency, equity, and quality.
  columns:
  - name: access_rank
    description: Ranking for access dimension of care.
  - name: cc_rank
    description: Ranking for care coordination.
  - name: cpsc_hco_id
    description: Unique identifier for the healthcare organization.
  - name: cpsc_mbr_key
    description: Unique key for the CPSC member.
  - name: cpsc_rank_key
    description: Key to group and track rank-based metrics.
  - name: dss_create_time
    description: '{{ doc(''dss_create_time'') }}'
  - name: dss_update_time
    description: '{{ doc(''dss_update_time'') }}'
  - name: dt_yr_key
    description: '{{ doc(''dt_yr_key'') }}'
  - name: eff_rank
    description: Ranking for efficiency of care delivery.
  - name: eq_rank
    description: Ranking for equity of access and care.
  - name: over

In [97]:
# === 📥 All new common_columns.md entries ===
if all_new_common_docs:
    print("\n\n=== 📥 New common_columns.md entries to add ===")
    for col, descs in sorted(all_new_common_docs.items()):
        # Filter out blank or templated doc strings
        clean_descs = [d for d in descs if d.strip() and not d.strip().startswith("{{ doc(")]
        if not clean_descs:
            continue  # Skip if nothing valid to print

        # Use first sorted valid definition (can be made smarter)
        definition = sorted(clean_descs)[0]
        print(f"\n{{% docs {col} %}}\n{definition}\n{{% enddocs %}}")


In [108]:
# === ⚠️ Conflicting definitions summary ===
if all_conflicts:
    print("\n\n=== ⚠️ Columns with conflicting definitions (needs manager input) ===")
    
    seen_columns = set()  # Track already printed columns

    for conflict in all_conflicts:
        col = conflict['column']
        if col in seen_columns:
            continue  # Skip already printed columns
        seen_columns.add(col)

        print(f"\nColumn: {col}")
        for inst in conflict["instances"]:
            print(f"  - Model: {inst['model']}")
            print(f"    File: {inst['file']}")
            print(f"    Definition:\n      \"{inst['definition']}\"\n")
        print("─" * 80)  # 👈 visual separator between columns
else:
    print("\n✅ No conflicting definitions found.")




=== ⚠️ Columns with conflicting definitions (needs manager input) ===

Column: service_line_descr
  - Model: dim_service_line_custom_qa
    File: C:\dbt-opt-automation-dbx\src\models\marts\cdb_dim\__cdb_dim_documentation.yml
    Definition:
      "Full description of the QA-defined service line (e.g., Oncology, Vascular Surgery, Pulmonary/Critical Care)."

  - Model: dim_service_line_custom_ban_phx
    File: C:\dbt-opt-automation-dbx\src\models\marts\cdb_dim\__cdb_dim_documentation.yml
    Definition:
      "Full description of the custom service line (e.g., Abdominal Transplant, Cardiology, Neurosurgery, Surgical Oncology)."

  - Model: dim_service_line_custom_ban_south
    File: C:\dbt-opt-automation-dbx\src\models\marts\cdb_dim\__cdb_dim_documentation.yml
    Definition:
      "Full description of the custom service line (e.g., Behavioral Health, Cardiology, General Medicine, Orthopedics, Trauma)."

  - Model: dim_service_line_custom_ban_tucs_ped
    File: C:\dbt-opt-automation-db

In [99]:
# === 📝 Placeholder doc blocks ===
if all_placeholders:
    print("\n\n=== 📝 Placeholder blocks for common_columns.md ===")
    seen_placeholders = set()
    for block in all_placeholders:
        col_match = re.search(r"{% docs ([^ ]+) %}", block)
        if col_match:
            col = col_match.group(1)
            if col in seen_placeholders:
                continue
            seen_placeholders.add(col)
        print("\n" + block)




=== 📝 Placeholder blocks for common_columns.md ===

# TODO: `service_line_descr` has conflicting definitions across 8 model(s).
# - Models involved: dim_service_line_custom_ban_phx, dim_service_line_custom_ban_south, dim_service_line_custom_ban_tucs, dim_service_line_custom_ban_tucs_ped, dim_service_line_custom_nov, dim_service_line_custom_qa, dim_service_line_custom_sg2, dim_service_line_custom_uvm
# - Definitions include:
#   - "Full description of the QA-defined service line (e.g., Oncology, Vascular Surgery, Pulmonary/Critical Care)."
#   - "Full description of the custom service line (e.g., Abdominal Transplant, Cardiology, Neurosurgery, Surgical Oncology)."
#   - "Full description of the custom service line (e.g., Behavioral Health, Cancer, Neurosciences, Surgery, Women’s Health)."
#   - "Full description of the custom service line (e.g., Behavioral Health, Cardiology, General Medicine, Orthopedics, Trauma)."
#   - "Full description of the custom service line (e.g., Bowel Obstr

In [100]:
import re

doc_ref_pattern = re.compile(r"\{\{\s*doc\(['\"]([^'\"]+)['\"]\)\s*\}\}")

used_doc_refs = set()
for model in full_schema_models:
    for col in model["columns"]:
        desc = col.get("description", "")
        for match in doc_ref_pattern.finditer(desc):
            used_doc_refs.add(match.group(1))

undefined_doc_refs = used_doc_refs - common_cols

if undefined_doc_refs:
    print("\n🚨 Unresolved doc() references (in YAML but not in common_columns.md):")
    for ref in sorted(undefined_doc_refs):
        print(f"  - {ref}")
else:
    print("\n✅ All doc() references in YAML are defined in common_columns.md.")



🚨 Unresolved doc() references (in YAML but not in common_columns.md):
  - service_line_descr


In [109]:
from collections import Counter

# === Build sets for analysis ===

# Columns appearing in executive tables (distinct)
exec_columns = set()
for tbl, cols in exec_cols.items():
    exec_columns.update(cols)


# Columns defined in common markdown
common_columns = common_cols


# Columns defined in other documentation YAMLs
other_doc_columns = set(column_sources.keys())

# How many exec columns are in common
exec_in_common = exec_columns & common_columns

# How many exec columns are in other docs
exec_in_other_docs = exec_columns & other_doc_columns

# Exec columns NOT in common or other docs
exec_unique_only = exec_columns - common_columns - other_doc_columns

# Count how many exec columns show up in multiple exec models
exec_col_counts = Counter()
for tbl, cols in exec_cols.items():
    for col in cols:
        exec_col_counts[col] += 1
# Which exec-only columns appear multiple times
exec_only_mult = {col for col in exec_unique_only if exec_col_counts[col] > 1}

# Which exec-only appear only once (likely inline descriptions)
exec_only_single = {col for col in exec_unique_only if exec_col_counts[col] == 1}

# === Print summary ===

print("\n=== 🧾 Column reuse and doc coverage stats for marts/executive ===\n")
print(f"Total distinct exec columns: {len(exec_columns)}")
print(f"In common_columns.md:          {len(exec_in_common)}")
print(f"In other documentation YAMLs:   {len(exec_in_other_docs)}")
print(f"Only in executive (unique):    {len(exec_unique_only)}")

print("\n--- Breakdown of exec‑only unique columns ---")
print(f"Exec‑only columns seen multiple times (candidates for centralization): {len(exec_only_mult)}")
print(f"Exec‑only columns seen once (inline descriptions):                   {len(exec_only_single)}")

print("\n=== Example Lists (truncated) ===\n")

print("Columns in common_columns.md (exec ∩ common):")
for c in sorted(list(exec_in_common))[:10]:
    print(f"  - {c}")
if len(exec_in_common) > 10: print(f"  ... +{len(exec_in_common) - 10} more\n")

print("Columns in other docs (exec ∩ other):")
for c in sorted(list(exec_in_other_docs))[:10]:
    print(f"  - {c}")
if len(exec_in_other_docs) > 10: print(f"  ... +{len(exec_in_other_docs) - 10} more\n")

print("Exec‑only columns seen multiple times (likely candidates):")
for c in sorted(list(exec_only_mult))[:10]:
    print(f"  - {c}")
if len(exec_only_mult) > 10: print(f"  ... +{len(exec_only_mult) - 10} more\n")

print("Exec‑only columns seen once (likely inline):")
for c in sorted(list(exec_only_single))[:10]:
    print(f"  - {c}")
if len(exec_only_single) > 10: print(f"  ... +{len(exec_only_single) - 10} more\n")



=== 🧾 Column reuse and doc coverage stats for marts/executive ===

Total distinct exec columns: 208
In common_columns.md:          39
In other documentation YAMLs:   26
Only in executive (unique):    168

--- Breakdown of exec‑only unique columns ---
Exec‑only columns seen multiple times (candidates for centralization): 47
Exec‑only columns seen once (inline descriptions):                   121

=== Example Lists (truncated) ===

Columns in common_columns.md (exec ∩ common):
  - Spend
  - capacity_order
  - compare_capacity
  - compare_descr
  - dss_create_time
  - dss_update_time
  - dt_mon_key
  - dt_qtr_key
  - dt_yr_key
  - focus_capacity
  ... +29 more

Columns in other docs (exec ∩ other):
  - Spend
  - capacity_order
  - compare_capacity
  - compare_descr
  - dss_create_time
  - dss_update_time
  - dt_mon_key
  - dt_qtr_key
  - dt_yr_key
  - focus_capacity
  ... +16 more

Exec‑only columns seen multiple times (likely candidates):
  - ELTBatchID
  - ELTExecutionDate
  - ELTSysCh

In [102]:
from pathlib import Path
EXEC_DOC_PATH = str(Path(EXEC_DOC_PATH).resolve())

ALL_DOCS = [str(Path(f).resolve()) for f in ALL_DOCS]

MARTS_DOCS_EXCLUDING_EXEC = [f for f in ALL_DOCS if f != EXEC_DOC_PATH]

print(MARTS_DOCS_EXCLUDING_EXEC)

['C:\\dbt-opt-automation-dbx\\src\\models\\marts\\app_insights_deltashare\\__ds_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\cdb_core\\__cdb_core_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\cdb_dim\\__cdb_dim_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\custom_sl\\__custom_sl_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\ice\\_ice_table_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\kaufman_hall\\_kh_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\odb\\_odb_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\outcomes\\__outcomes_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\pa_dim\\_pa_capacity_throughput_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\qa_readmits\\__qa_readmit_documentation.yml', 'C:\\dbt-opt-automation-dbx\\src\\models\\marts\\solution_agnostic_data_health\\_solution_agn